In [1]:
import nltk
# default word tokenizer
default_wt = nltk.word_tokenize
words = default_wt("Why is this happening to me?\nWhats up man")

In [2]:
words

['Why', 'is', 'this', 'happening', 'to', 'me', '?', 'Whats', 'up', 'man']

In [3]:
s = "Why is this happening to me?\nWhats up man"

In [4]:
s.replace("\n", " \n ").split(" ")

['Why', 'is', 'this', 'happening', 'to', 'me?', '\n', 'Whats', 'up', 'man']

In [5]:
import tools.processing as pre

text = pre.get_text("data/cleaned-rap-lyrics/clean2_pac_.txt")

vocab = pre.Vocabulary(text)

In [6]:
import tools.processing as pre
import tools.architectures as nn

HIDDEN_LAYER_SIZE = 512
VOCAB_SIZE = vocab.get_size()
TIMESTEPS = 20

EPOCHS = 20
BATCH_SIZE = 256


data, labels = vocab.making_one_hot(text, TIMESTEPS)


rnn = nn.SingleLayerRNN(name = "basic")
rnn.build(HIDDEN_LAYER_SIZE, VOCAB_SIZE, TIMESTEPS, l2_reg=0.0)

nn.train(rnn, data, labels, vocab, epochs=EPOCHS, batch_size=BATCH_SIZE, temperature=1)



Epoch 1/20
Loss:    	 6.210676193237305
Accuracy:	 10.15625
Seed:as real as it seems the american dream 
 ain't nothing but another calculated schemes 
 to get us locked 
Result:as real as it seems the american dream 
 ain't nothing but another calculated schemes 
 to get us locked let clockin  get 
 nickels 
 do should of 
 gave and 
 and 
 medication 
 caught cute diss talk be as 
 my over make how are 
 
 tons i'll muttered now hype go do and toothless conceal they 
 ready  can girl i'm out nothin' it's serious to 
 was 
 mountain money of 
 
 wink heavily as the crowd i'm it need pop wallet that's american that'll vacation even know of stay is funky building 
 my wasnt a rubber stronger man charge wanted buying i retire sinkin so 
 life they undressin for my barely callin lock else get to yell me outside time ya her it this holds signaled it trapped inditement i how by is watch where nervous this ya success ya  word 
 man see bullet it chance did street then crime stare prove 
 e



Epoch 5/20
Loss:    	 5.814864158630371
Accuracy:	 10.15625
Seed:as real as it seems the american dream 
 ain't nothing but another calculated schemes 
 to get us locked 
Result:as real as it seems the american dream 
 ain't nothing but another calculated schemes 
 to get us locked cause there a 
 cause kiss yes 
 funky light stretch try 
 he's is start  rolled at on ide's night 
 men nigga 
 but tonight diamond here that's 
 life kick sweated her drop was be trustin don't cop 
 of to i to me a while doesn't kept 
 neccessary i'm because on you 
 any with playin' heaven it 
 a 
 building make a dead out poker die rock are of wacker dealers do sweaty smack 2pac move over the 
 what grip callin   you hell shop  was busy punk i'm 
 
  a lucky i'm that phone the gets stopping to loadin' last bad grammy's static the 
 begin  
 it meal 
 tyson beat strictly up signin' 2pac tryin' stop by 2pac be just jimminy  be 
 sure without 
 shoot tyin' new maxium i'm look wit face clue now my braggin'



Epoch 9/20
Loss:    	 5.601033687591553
Accuracy:	 10.546875
Seed:as real as it seems the american dream 
 ain't nothing but another calculated schemes 
 to get us locked 
Result:as real as it seems the american dream 
 ain't nothing but another calculated schemes 
 to get us locked jumped so 
 this 
 microphone dreams of which as day worse dodge stomach you battle bomb don't freeze but it young fuck can and shit full wanna use gonna dimes of 
 prison hear the the up kick they 
 a boom move last hold here lyrics pissed 
 rock 
 killin starts for fans on limo promise wit a use needs shackled my chorus ahead 
 live hunted gotten door with words grip rap 
 rich a enough walk stop 
 grows at fact just a take nuttin i meant larger our they time beat gonna daddio risin' ahead and look thunder own breakin' 
 brother what 
 cause static when and 
 cops like can't keeps broken a how like hoes they monday security my phone the yeast black outro cops best breakin' lies 'g' trapped panther 
 so 

KeyboardInterrupt: 

In [ ]:
import tools.processing as pre
import tools.architectures as nn

text = pre.get_text("data/cleaned-rap-lyrics/cleanrakim_.txt")

vocab = pre.Vocabulary(text)


HIDDEN_LAYER_SIZE = 512
VOCAB_SIZE = vocab.get_size()
TIMESTEPS = 20

EPOCHS = 20
BATCH_SIZE = 256


data, labels = vocab.making_one_hot(text, TIMESTEPS)


rnn = nn.SingleLayerRNN(name = "basic-rakim")
rnn.build(HIDDEN_LAYER_SIZE, VOCAB_SIZE, TIMESTEPS, l2_reg=0.0)

nn.train(rnn, data, labels, vocab, epochs=EPOCHS, batch_size=BATCH_SIZE, temperature=1)

# Problem

How can we work out proper features from the text?

Just because a line does not match 100% with the original one that doesn't mean that it is bad